### Imports + utility funcitons

In [1]:
import json
import geopandas as gpd
import numpy as np
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt

from pystac_client import Client
import stackstac

In [2]:
# Utility functions
def get_bbox(shp_fp):
    gdf = gpd.read_file(shp_fp)
    if gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(4326)
    bbox = tuple(gdf.total_bounds)

    return bbox

### Work through STAC catalog to find product URLs

In [3]:
# Use City of Ottawa urban boundary as test AOI
shp_fp = './Urban_Boundary.geojson'
aoi_gdf = gpd.read_file(shp_fp)
utm_crs = aoi_gdf.estimate_utm_crs().to_epsg()  # required for stackstac call
bbox = get_bbox(shp_fp)
bbox

(-75.96666401487873, 45.23090774120208, -75.43410167132566, 45.516074722660285)

In [4]:
# Open EODMS STAC catalog and explore available collections
cat_url = 'https://www.eodms-sgdot.nrcan-rncan.gc.ca/stac/'
catalog = Client.open(cat_url)
collections = list(catalog.get_collections())

print(f"Number of collections: {len(collections)}")
print("Collection IDs:")
for collection in collections:
    print(f"- {collection.id}")

Number of collections: 3
Collection IDs:
- sentinel-1
- rcm-ard
- rcm


In [5]:
# Get RCM-CEOS-ARD collection by ID and explore
collection = catalog.get_child('rcm-ard')
collection

<CollectionClient id=rcm-ard>

In [6]:
# List all RCM-ARD items in collection that intersect with area of interest
search = catalog.search(
    collections=["rcm-ard"],
    bbox=bbox
)

items = search.item_collection()
print(f"Returned {len(items)} Items:")
for item in items:
    print(item.properties['title'])

Returned 4 Items:
RCM1_OK2978586_PK3061982_1_SC30MCPC_20240511_225201_CH_CV_MLC
RCM2_OK2603500_PK2843058_1_SC30MCPC_20231123_111136_CH_CV_MLC
RCM2_OK1451009_PK1451245_2_5M12_20200424_111154_HH_HV_SLC
RCM3_OK2017056_PK2017214_1_16MCP3_20200421_224405_CH_CV_SLC


In [7]:
# Filter results by desired properties: looking for MLC products with 'CH', 'CV', and 'XC' polarizations
polarizations = ['CH', 'CV', 'XC']
product_type = 'MLC'
filtered_items = [x for x in items if x.properties['sar:polarizations'] == polarizations and x.properties['sar:product_type'] == product_type]

RCM1_OK2978586_PK3061982_1_SC30MCPC_20240511_225201_CH_CV_MLC Medium Resolution 30m
RCM2_OK2603500_PK2843058_1_SC30MCPC_20231123_111136_CH_CV_MLC Medium Resolution 30m


In [28]:
# Get S3 URLs for all cloud-optimized GeoTIFFs needed for M-CHI decomposition
card_product = filtered_items[0]
rl_url, rr_url, rrrl_url = card_product.assets['RL'].href, card_product.assets['RR'].href, card_product.assets['RRRL'].href
rl_url, rr_url, rrrl_url

('https://rcm-ceos-ard.s3.ca-central-1.amazonaws.com/MLC/2024/05/11/RCM1_OK2978586_PK3061982_1_SC30MCPC_20240511_225201_CH_CV_MLC/RCM1_OK2978586_PK3061982_1_SC30MCPC_20240511_225201_RL.tif',
 'https://rcm-ceos-ard.s3.ca-central-1.amazonaws.com/MLC/2024/05/11/RCM1_OK2978586_PK3061982_1_SC30MCPC_20240511_225201_CH_CV_MLC/RCM1_OK2978586_PK3061982_1_SC30MCPC_20240511_225201_RR.tif',
 'https://rcm-ceos-ard.s3.ca-central-1.amazonaws.com/MLC/2024/05/11/RCM1_OK2978586_PK3061982_1_SC30MCPC_20240511_225201_CH_CV_MLC/RCM1_OK2978586_PK3061982_1_SC30MCPC_20240511_225201_RRRL.tif')

### Read data from COGs using spatial subset

RRRL GeoTiff contains 2 bands (real + imaginary) so we define our own function instead of using stackstac, which only supports single-band rasters

In [119]:
def read_cog_from_bbox(cog_url, bbox):
    with rxr.open_rasterio(cog_url) as rds:
        # Reproject bounding box to match raster CRS
        bbox_geom = box(*bbox)
        cog_epsg = rds.rio.crs.to_epsg()
    
        project = pyproj.Transformer.from_proj(
            pyproj.Proj('epsg:4326'),
            pyproj.Proj(f'epsg:{cog_epsg}'),
            always_xy = True) 
        
        bbox_geom_proj = transform(project.transform, bbox_geom)
    
        # Clip COG to reprojected bounds and return
        out_cog = rds.rio.clip_box(*bbox_geom_proj.bounds)
    
    return out_cog

In [123]:
rl, rr, rrrl =  read_cog_from_bbox(rl_url, bbox), read_cog_from_bbox(rr_url, bbox), read_cog_from_bbox(rrrl_url, bbox)